In [3]:
from sklearn.decomposition import DictionaryLearning

#importando bibliotecas
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from IPython.display import Image
from pathlib import Path


config_dirs = open("prefixo_notebook.txt").readlines() 
DIR_PREFIXO = Path(config_dirs[0].strip())


DIRETORIO_PRINCIPAL =  DIR_PREFIXO / "Projeto_PA" / "Projeto_PA_validado"
DIR_MODIFICADO = DIRETORIO_PRINCIPAL / "dataset_modificado"
DIR_NMF = DIRETORIO_PRINCIPAL / "DICT"
DIRETORIO_DATASET = DIR_PREFIXO / "Projeto_PA"

DIR_KMEANS = DIRETORIO_PRINCIPAL / "KMeansRandom"


In [4]:
# Datset com todos os campos: metadados, medidas e estatísticas

dataset = pd.read_csv(DIR_KMEANS / "padrao_doenca" / "pacientes_aceitos_apos_tratamento_dados.csv", sep=";")


#dataset = pd.read_csv('pacientes_inputados_e_nao_descartados_sample.csv',sep=';')
#dataset = dataset[dataset['Descartado']==False]
#trocar nulo por 0
dataset_filtrado = dataset.fillna(0)
#Selecionar as que serao utilizadas
dataset_filtrado_tratado = dataset_filtrado.iloc[:,8:172]


In [5]:
#numero de componentes do Dictionary Learning
num_comp = 5

In [6]:
#parametros da tecnica
dict_learner = DictionaryLearning(n_components=num_comp)

In [7]:
#pegando as variaveis 
x_variaveis = dataset_filtrado_tratado.values

In [8]:
#aplicando o dictionaryLearning
X_transformed = dict_learner.fit_transform(x_variaveis)

KeyboardInterrupt: 

In [ ]:
#resultado
X_transformed

In [ ]:
#Funções para calcular com K-Means o melhor numero de cluster(grupos) que seu dataset gerar
def calculate_wcss(data):
    """
    Calcula a soma dos quadrados intra-clusters para 19
    quantidades de clusters, iniciando com o mínimo de 2 clusters
    
    Parametros
    ----------
    data : DataFrame
        conjunto de dados para fazer o `.fit()` do KMeans

    
    Returns
    -------
    wcss : lista contendo os valores de soma de quadrados intra-cluster
    """
    from sklearn.cluster import KMeans
    wcss = []
    for n in range(2, 21):
        kmeans = KMeans(n_clusters=n)
        kmeans.fit(X=data)
        wcss.append(kmeans.inertia_)

    return wcss

def optimal_number_of_clusters(wcss):
    """
    Calcula a maior distância entre os pontos que marcam as 
    somas dos quadrados intra-clusters para 19 calculadas 
    com `calculate_wcss()`
    
    Parametros
    ----------
    wcss : lista
        lista contendo os valores de soma de quadrados intra-cluster

    
    Returns
    -------
    int : número de clusters 
    """
    from math import sqrt
    x1, y1 = 2, wcss[0]
    x2, y2 = 20, wcss[len(wcss)-1]

    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]

        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    return distances.index(max(distances)) + 2

In [ ]:
#calculando a quantidade ótima de clusters (é possivel dividir os pacientes em quantos grupos)
sum_of_squares = calculate_wcss(X_transformed)
n = optimal_number_of_clusters(sum_of_squares)
print(n)

In [ ]:
#habilitando o kmeans com n grupos (foi calculado na etapa anterior)
kmeans = KMeans(n_clusters=n, random_state=0).fit(X_transformed)

In [ ]:
#lista de grupos 
grupos = kmeans.fit_predict(x_variaveis)

In [ ]:
#colocar os grupos gerados apartir do DictionaryLearning
dataset_filtrado['grupo'] = grupos 

In [ ]:
dataset_filtrado

In [ ]:
dataset_filtrado.to_csv(´resultados´)

In [ ]:
#criar dataframe de contagem
dt_final = dataset_filtrado.groupby(['grupo']).count()

In [ ]:
# Dados para o plot
labels = list(dt_final.index)
sizes = list(dt_final['numero_identificacao'])

ls_itens = list(dt_final['numero_identificacao'])
item = max(ls_itens)
posicao = ls_itens.index(item)

lst_tlp = []
for i in range(0,len(ls_itens)):
    if i == posicao:
        lst_tlp.append(0.1)
    else:
        lst_tlp.append(0)

explode = tuple(lst_tlp) 

# Plot
plt.pie(sizes, explode=explode, labels=labels, 
autopct='%1.1f%%', shadow=True, startangle=140)

plt.axis('equal')
plt.show()